In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import warnings
warnings.filterwarnings('ignore')

In [79]:
from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm
from DLAIUtils import Utils

import ast
import os
import pandas as pd

### Setup pinecone

In [80]:
# get api key
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

pinecone = Pinecone(api_key=PINECONE_API_KEY)

In [81]:
utils = Utils()
INDEX_NAME = utils.create_dlai_index_name('dl-ai')
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

In [82]:
pinecone.create_index(name=INDEX_NAME, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [83]:
INDEX_NAME

'dl-ai-j532omwwt3blbkfjv9hy1uwtzo8q3errx4vh'

### Store embeddings

In [90]:
data_csv = "E:\data\wiki.csv"

In [91]:
max_articles_num = 2000
df = pd.read_csv(data_csv, nrows=max_articles_num)
df.head()

,id,metadata,values
1,1-0,"{'chunk': 0, 'source': 'https://simple.wikiped...","[-0.011254455894231796, -0.01698738895356655, ..."
2,1-1,"{'chunk': 1, 'source': 'https://simple.wikiped...","[-0.0015197008615359664, -0.007858820259571075..."
3,1-2,"{'chunk': 2, 'source': 'https://simple.wikiped...","[-0.009930099360644817, -0.012211072258651257,..."
4,1-3,"{'chunk': 3, 'source': 'https://simple.wikiped...","[-0.011600767262279987, -0.012608098797500134,..."
5,1-4,"{'chunk': 4, 'source': 'https://simple.wikiped...","[-0.026462381705641747, -0.016362832859158516,..."


In [92]:
ast.literal_eval(df.iloc[0].metadata)

{'chunk': 0,
 'source': 'https://simple.wikipedia.org/wiki/April',
 'text': "April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April

In [93]:
len(ast.literal_eval(df.iloc[1].metadata)['text'])

1546

In [94]:
len(ast.literal_eval(df.iloc[0]['values']))

1536

In [95]:
df.columns, df.shape

(Index(['id', 'metadata', 'values'], dtype='object'), (2000, 3))

In [96]:
prepped = []

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    meta = ast.literal_eval(row['metadata'])
    values = ast.literal_eval(row['values'])
    
    prepped.append({'id':row['id'], 
                    'values':values, 
                    'metadata':meta})
    if len(prepped) >= 250:
        index.upsert(prepped)
        prepped = []


100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [00:47<00:00, 42.17it/s]


In [32]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2000}},
 'total_vector_count': 2000}

### Connect to OpenAI

In [44]:
OPENAI_API_KEY = utils.get_openai_api_key()
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [46]:
def get_embeddings(articles, model="text-embedding-ada-002"):
   return openai_client.embeddings.create(input = articles, model=model)

### Embedd query

In [47]:
query = "what is the berlin wall?"

embed = get_embeddings([query])
res = index.query(vector=embed.data[0].embedding, top_k=3, include_metadata=True)
text = [r['metadata']['text'] for r in res['matches']]
print('\n'.join(text))

After World War II 

After World War II, Germany was left in ruins. The victorious Allies that occupied it split it into four parts. in the western half of Germany, one part was given to the United States, one to the United Kingdom, and one to France. The eastern half was occupied by the USSR. The city of Berlin was also split among the four countries even though it was entirely within the eastern half.

The Federal Republic of Germany (Bundesrepublik Deutschland or BRD), or West Germany, was recognized by the Western Allies in June 1949 and was a capitalist democracy. West Berlin was considered a part of the country. The Soviets named their section of Germany the German Democratic Republic (Deutsche Demokratische Republik or DDR), or East Germany, later in 1949; it was a communist dictatorship.

From April 1948 to May 1949, the Soviets blockaded West Berlin to prevent the city from using West Germany's currency. The United States and its allies supplied the city by airplanes until Sep

In [74]:
len(res.matches)

3

In [75]:
res.matches[0]

{'id': '1949-2',
 'metadata': {'chunk': 2.0,
              'source': 'https://simple.wikipedia.org/wiki/Cold%20War',
              'text': 'After World War II \n'
                      '\n'
                      'After World War II, Germany was left in ruins. The '
                      'victorious Allies that occupied it split it into four '
                      'parts. in the western half of Germany, one part was '
                      'given to the United States, one to the United Kingdom, '
                      'and one to France. The eastern half was occupied by the '
                      'USSR. The city of Berlin was also split among the four '
                      'countries even though it was entirely within the '
                      'eastern half.\n'
                      '\n'
                      'The Federal Republic of Germany (Bundesrepublik '
                      'Deutschland or BRD), or West Germany, was recognized by '
                      'the Western Allies 

### Build prompt

In [97]:
query = "write an article titled: what is the berlin wall?"
embed = get_embeddings([query])
res = index.query(vector=embed.data[0].embedding, top_k=3, include_metadata=True)

contexts = [
    x['metadata']['text'] for x in res['matches']
]

prompt_start = (
    "Answer the question based on the context below.\n\n"+
    "Context:\n"
)

prompt_end = (
    f"\n\nQuestion: {query}\nAnswer:"
)

prompt = (
    prompt_start + "\n\n---\n\n".join(contexts) + 
    prompt_end
)

print(prompt)

Answer the question based on the context below.

Context:
After World War II 

After World War II, Germany was left in ruins. The victorious Allies that occupied it split it into four parts. in the western half of Germany, one part was given to the United States, one to the United Kingdom, and one to France. The eastern half was occupied by the USSR. The city of Berlin was also split among the four countries even though it was entirely within the eastern half.

The Federal Republic of Germany (Bundesrepublik Deutschland or BRD), or West Germany, was recognized by the Western Allies in June 1949 and was a capitalist democracy. West Berlin was considered a part of the country. The Soviets named their section of Germany the German Democratic Republic (Deutsche Demokratische Republik or DDR), or East Germany, later in 1949; it was a communist dictatorship.

From April 1948 to May 1949, the Soviets blockaded West Berlin to prevent the city from using West Germany's currency. The United Stat

### RAG

In [98]:
res = openai_client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature=0,
    max_tokens=636,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print('-' * 80)
print(res.choices[0].text)

--------------------------------------------------------------------------------

The Berlin Wall, also known as the "Iron Curtain," was a physical barrier that divided the city of Berlin from 1961 to 1989. It was built by the communist government of East Germany to prevent its citizens from fleeing to the democratic West Germany. The wall was a symbol of the Cold War and the division between the Western and Eastern powers.

The construction of the Berlin Wall began on August 13, 1961, and it consisted of a concrete wall, barbed wire, and guard towers. The wall was 96 miles long and 12 feet high, with a "death strip" in between the two sides. This strip was heavily guarded and contained mines, dogs, and other obstacles to prevent people from crossing.

The Berlin Wall was a result of the tensions between the Soviet Union and the Western Allies after World War II. Germany was divided into four parts, with the Western Allies occupying the western half and the Soviet Union occupying the e